# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [3]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from rs_datasets import MovieLens

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic import metrics

from sponge_bob_magic.models.pop_rec import PopRec
from sponge_bob_magic.models.als_rec import ALSRec
from sponge_bob_magic.models.knn_rec import KNNRec
from sponge_bob_magic.models.lightfm_rec import LightFMRec

from sponge_bob_magic.scenarios.main_scenario import MainScenario
from sponge_bob_magic.session_handler import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [4]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [5]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [6]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [7]:
pop_rec = PopRec()
als_rec = ALSRec()
knn_rec = KNNRec()
lightfm_rec = LightFMRec()

In [8]:
log_bydate_splitter = log_splitter.DateSplitter(
    test_start=datetime(2007, 1, 1),
    drop_cold_items=True,
    drop_cold_users=True
)
log_random_splitter = log_splitter.RandomSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)
log_cold_splitter = log_splitter.ColdUsersSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True
)
user_random_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
    
)
user_bydate_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)

### Обучение сценария <a name="fit-scenario"></a>

In [9]:
results = None

In [10]:
scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=lightfm_rec,
    criterion=metrics.HitRate,
    metrics={
        metrics.NDCG: [10,5,3],
        metrics.Precision: [10,5,3],
        metrics.MAP: [10,5,3],
        metrics.Recall: [10,5,3],
        metrics.Surprisal: [10,5,3],
    },
    fallback_rec=pop_rec,
)

In [11]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [12]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

03-Mar-20 16:36:51, root, DEBUG: Деление лога на обучающую и тестовую выборку
03-Mar-20 16:37:11, root, DEBUG: Длина трейна и теста: (84013, 15944)
03-Mar-20 16:37:12, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490
03-Mar-20 16:37:13, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355
03-Mar-20 16:37:13, root, DEBUG: Инициализация метрик
03-Mar-20 16:37:14, root, DEBUG: Обучение и предсказание дополнительной модели
03-Mar-20 16:37:14, root, DEBUG: Проверка датафреймов
03-Mar-20 16:37:14, root, DEBUG: Предварительная стадия обучения (pre-fit)
03-Mar-20 16:37:15, root, DEBUG: Среднее количество items у каждого user: 90
03-Mar-20 16:37:17, root, DEBUG: Основная стадия обучения (fit)
03-Mar-20 16:37:17, root, DEBUG: Проверка датафреймов
03-Mar-20 16:37:20, root, DEBUG: Количество items после фильтрации: 100
03-Mar-20 16:37:23, root, DEBUG: Пре-фит модели
03-Mar-20 16:37:23, root, DEBUG: -------------
03-Mar-20 16:37:23, root, DEBUG: Оптимизация параметров
03-Ma

In [13]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

number                state     value             datetime_start  \
                                                                     
0  0      TrialState.COMPLETE  0.863265 2020-03-03 16:37:24.013925   
1  1      TrialState.COMPLETE  0.846939 2020-03-03 16:38:20.652738   

           datetime_complete params  \
                               rank   
0 2020-03-03 16:38:20.563446  16      
1 2020-03-03 16:39:15.889279  91      

                                                                 user_attrs  \
                                                                        MAP   
0  {5: 0.46341836734693914, 10: 0.5296129213779236, 3: 0.41734693877551077}   
1  {5: 0.41331972789115695, 10: 0.4770973616960013, 3: 0.36768707482993246}   

                                                                              \
                                                                   Precision   
0  {5: 0.2355102040816325, 10: 0.23551020408163278, 3: 0.24761904761904754}    
1  {5: 0.22367346938775512, 10: 0.22857142857142848, 3: 0.22789115646258495}   

                                                                               \
                                                                       Recall   
0  {5: 0.054368928645132375, 10: 0.10249637765615235, 3: 0.0335897875976151}    
1  {5: 0.05100607467392562, 10: 0.09727026066787188, 3: 0.031314565352585186}   

                                                                              \
                                                                   Surprisal   
0  {5: 0.16283880795451958, 10: 0.17683063667039753, 3: 0.15427899488448096}   
1  {5: 0.18132387309566583, 10: 0.19505678411531838, 3: 0.17494870907613017}   

                                                                             \
                                                                       nDCG   
0  {5: 0.24662745690036733, 10: 0.2509328100943388, 3: 0.25758809410065275}   
1  {5: 0.2290539920045391, 10: 0.2372606863386784, 3: 0.23331455504591966}    

  system_attrs  
       _number  
0  0            
1  1

### Получение рекомендаций <a name="predict-scenario"></a>

In [14]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

03-Mar-20 16:39:15, root, DEBUG: Проверка датафреймов
03-Mar-20 16:39:16, root, DEBUG: Предварительная стадия обучения (pre-fit)
03-Mar-20 16:39:16, root, DEBUG: Основная стадия обучения (fit)
03-Mar-20 16:39:16, root, DEBUG: Построение модели LightFM
03-Mar-20 16:39:18, root, DEBUG: Проверка датафреймов
03-Mar-20 16:39:18, root, DEBUG: Выделение дефолтных юзеров
03-Mar-20 16:39:18, root, DEBUG: Выделение дефолтных айтемов


In [15]:
recs.show()

+-------+-------+--------------------+----------+
|user_id|item_id|           relevance|   context|
+-------+-------+--------------------+----------+
|     91|    199|-0.00623861979693...|no_context|
|     91|    174| -0.0776733011007309|no_context|
|     91|    511|-0.08005917072296143|no_context|
|     91|    187|-0.21621178090572357|no_context|
|     91|    300| -0.2601206600666046|no_context|
|     91|    435|-0.38647374510765076|no_context|
|     91|    172| -0.4527072012424469|no_context|
|     91|    205|-0.45428895950317383|no_context|
|     91|    197| -0.4561214745044708|no_context|
|     91|    194|-0.45944446325302124|no_context|
|    152|     88|  0.5004076957702637|no_context|
|    152|    393| 0.20400138199329376|no_context|
|    152|     66| 0.18721534311771393|no_context|
|    152|     15|0.029790040105581284|no_context|
|    152|    111|-0.05663729831576...|no_context|
|    152|    237|-0.10480618476867676|no_context|
|    152|    216| -0.3903554379940033|no_context|
